In [ ]:
import torch
from torch import nn 
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('heart.csv')
df["Sex"].replace({'M': 0, 'F': 1}, inplace=True)
df["ChestPainType"].replace({"ATA": 0, "NAP": 1, "ASY": 2, "TA": 3}, inplace=True)
df["RestingECG"].replace({"Normal": 0, "ST": 1, "LVH": 2}, inplace=True)
df["ExerciseAngina"].replace({'N': 0, 'Y': 1}, inplace=True)
df["ST_Slope"].replace({"Up": 0, "Flat": 1, "Down": 2}, inplace=True)

In [ ]:
features = ['ST_Slope',
 'ChestPainType',
 'ExerciseAngina',
 'Cholesterol',
 'MaxHR',
 'Oldpeak',
 'Sex',
 'FastingBS',
 'Age',
 'RestingBP']

In [ ]:
X, x_test, Y, y_test = train_test_split(df[features], df['HeartDisease'], train_size=0.8, test_size=0.2, random_state=42)
X_train, x_valid, Y_train, y_valid = train_test_split(X,Y, test_size=0.25, train_size=0.75)

In [ ]:
#train => 60%, test=> 20%, valid => 20%
X_train.shape, Y_train.shape, x_test.shape, x_valid.shape

In [ ]:
class STPDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data 
    
    def __len__ (self):
        return len(self.X_data)
    
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

In [ ]:
train_data = STPDataset(torch.FloatTensor(X_train.values), torch.FloatTensor(Y_train.values))
test_data = STPDataset(torch.FloatTensor(x_test.values), torch.FloatTensor(y_test.values))
valid_data = STPDataset(torch.FloatTensor(x_valid.values), torch.FloatTensor(y_valid.values))

In [ ]:
# Create data loaders
train_loader = DataLoader(train_data, batch_size=9, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=1)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

In [ ]:
len(train_loader), len(test_loader)

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

In [ ]:
class HeartNet(nn.Module):
    
    def __init__(self, input_dim, output_dim, hidden_dim, dropout_prob):
        super(HeartNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_prob)
        self.activation = nn.ReLU()
    
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.activation(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x 

In [ ]:
HeartNN = HeartNet(10, 1, 96, 0.08610917049085276)

In [ ]:
HeartNN

In [ ]:
criterion = nn.BCEWithLogitsLoss() #multiple feature use binary cross entropy loss
optimizer = Adam(HeartNN.parameters(), lr=0.004773449431112121)
EPOCHS = 500

In [ ]:
sample = iter(train_loader).__next__() 
sample[0].shape, sample[1].shape

In [ ]:
sample[0], sample[1]

In [ ]:
len(train_loader)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

early_stopping_patience = 15
early_stopping_counter = 0

train_acc = []
train_loss = []

valid_acc = []
valid_loss = []

total_step = len(train_loader)
total_step_val = len(valid_loader)

valid_loss_min=np.inf

for epoch in range(EPOCHS):
    
    running_loss=0
    correct=0
    total=0
    
    #TRAINING

    HeartNN.train()

    for batch_idx, (X_train_batch, y_train_batch) in enumerate(train_loader):
        X_train_batch, y_train_batch = X_train_batch.to(DEVICE), y_train_batch.to(DEVICE)
        optimizer.zero_grad()
        output = HeartNN(X_train_batch)
        y_pred = torch.round(torch.sigmoid(output))
        #LOSS
        loss = criterion(output, y_train_batch.unsqueeze(1))
        writer.add_scalar("Loss/train", loss, epoch)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item() #sum loss for every batch
        #ACCURACY
        correct += torch.sum(y_pred==y_train_batch.unsqueeze(1)).item()
        writer.add_scalar("Accuracy/train", correct, epoch)
        total += y_train_batch.size(0)
    train_acc.append(100 * correct / total) #calculate accuracy among all entries in the batches
    train_loss.append(running_loss/total_step)  #get average loss among all batches dividing total loss by the number of batches

    # VALIDATION
    correct_v = 0
    total_v = 0
    batch_loss = 0
    with torch.no_grad():
        HeartNN.eval()
        for batch_idx, (X_valid_batch, y_valid_batch) in enumerate(valid_loader):
            X_valid_batch,y_valid_batch=X_valid_batch.to(DEVICE),y_valid_batch.to(DEVICE)
            #PREDICTION
            output = HeartNN(X_valid_batch)
            y_pred = torch.round(torch.sigmoid(output))
            #LOSS
            loss_v = criterion(output, y_valid_batch.unsqueeze(1))
            writer.add_scalar("Loss/validation", correct, epoch)
            batch_loss+=loss_v.item()
            #ACCURACY
            correct_v += torch.sum(y_pred==y_valid_batch.unsqueeze(1)).item()
            writer.add_scalar("Accuracy/validation", correct, epoch)
            total_v += y_valid_batch.size(0)
        valid_acc.append(100 * correct_v / total_v) 
        valid_loss.append(batch_loss/total_step_val)
    
    
    if np.mean(valid_loss) <= valid_loss_min:
        torch.save(HeartNN.state_dict(), './state_dict.pt')
        print(f'Epoch {epoch + 0:01}: Validation loss decreased ({valid_loss_min:.6f} --> {np.mean(valid_loss):.6f}).  Saving model ...')
        valid_loss_min = np.mean(valid_loss)
        early_stopping_counter=0 #reset counter if validation loss decreases
    else:
        print(f'Epoch {epoch + 0:01}: Validation loss did not decrease')
        early_stopping_counter+=1

    if early_stopping_counter > early_stopping_patience:
        print('Early stopped at epoch :', epoch)
        break

    print(f'\t Train_Loss: {np.mean(train_loss):.4f} Train_Acc: {(100 * correct / total):.3f} Val_Loss: {np.mean(valid_loss):.4f}  BEST VAL Loss: {valid_loss_min:.4f}  Val_Acc: {(100 * correct_v / total_v):.3f}\n')
writer.flush()
writer.close()

In [ ]:
y_pred_prob_list = []
y_pred_list = []


# Loading the best model
HeartNN.load_state_dict(torch.load('./state_dict.pt'))

with torch.no_grad():
	HeartNN.eval()
	for batch_idx, (X_test_batch, y_test_batch) in enumerate(test_loader):
		X_test_batch = X_test_batch.to(DEVICE)
		#PREDICTION
		output = HeartNN(X_test_batch)
		y_pred_prob = torch.sigmoid(output)
		y_pred_prob_list.append(y_pred_prob.cpu().numpy())
		y_pred = torch.round(y_pred_prob)
		y_pred_list.append(y_pred.cpu().numpy())

In [ ]:
y_pred_prob_list = [a.squeeze().tolist() for a in y_pred_prob_list]
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
print(classification_report(y_test, y_pred_list))

In [ ]:
accuracy_score(y_test, y_pred_list)

In [ ]:
precision_score(y_test, y_pred_list)

In [ ]:
recall_score(y_test, y_pred_list)

In [ ]:
f1_score(y_test, y_pred_list)

In [ ]:
confusion_matrix(y_test, y_pred_list)

In [ ]:
import matplotlib.pyplot as plt 

plt.plot(train_acc)

In [ ]:
plt.plot(train_loss)

In [ ]:
plt.plot(valid_acc)

In [ ]:
plt.plot(valid_loss)

In [ ]:
import onnx
import onnxruntime

HeartNN = HeartNet(10, 1, 96, 0.08610917049085276)

In [ ]:
HeartNN.load_state_dict(torch.load("state_dict.pt"))

In [ ]:
input_shape = (1, 10)

In [ ]:
import torch
dummy_input = torch.randn(input_shape)
output_path = "HeartNet.onnx"
torch.onnx.export(HeartNN, dummy_input, output_path, verbose=True)

In [ ]:
import onnx
import onnxruntime
onnx_model = onnx.load(output_path)
onnx.checker.check_model(onnx_model)

In [ ]:
input = [[  1.0000,   2.0000,   1.0000, 217.0000, 110.0000,   2.5000,   0.0000,
            0.0000,  55.0000, 158.0000]]

In [ ]:
import onnxruntime as ort

model_path = 'HeartNet.onnx'
session = ort.InferenceSession(model_path)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

In [ ]:
input_name

In [ ]:
output = session.run([output_name], {input_name: input})[0]

In [ ]:
output

In [ ]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
np.round(sigmoid(output))